# Plot1 Gender Pay Gap

In [1]:
#| echo: false
#| include: false
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import plotly.offline as pyo
import plotly.tools as tls
from plotly.graph_objects import Layout
from plotly.subplots import make_subplots
import numpy as np


import warnings
warnings.filterwarnings('ignore')

pio.renderers.default = "plotly_mimetype+notebook_connected"

In [2]:


import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots

label = None
k=0

data = pd.read_excel("../../data/cleaned/Gender_Pay_Gap.xlsx", sheet_name="US short version")

Category = data['Category'].unique()[:8]

data_sub = data[~(data['Occupation'].str.lower() == data['Category'].str.lower())]
data_sub = data_sub.drop(data_sub.index[-1])

def Figure(label, k):
    fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'polar'}, {'type': 'bar'}]],
                        column_widths=[0.2, 0.1])

    buttons = []
    for i in range(len(Category)):
        visible_indicator = False
        if i == k:
            visible_indicator = True

        occupation = data_sub[data_sub['Category'] == Category[i]]['Occupation'].values
        pay_gap_percent = data_sub[data_sub['Category'] == Category[i]]['Pay gap as a percentage'].values
        avg_salary_women = data_sub[data_sub['Category'] == Category[i]]['Women average annual salary ($)'].values
        avg_salary_men = data_sub[data_sub['Category'] == Category[i]]['Men average annual salary ($)'].values
        fig.add_trace(
            go.Bar(
                x=occupation, y=pay_gap_percent, name=Category[i], visible=visible_indicator, base='stack',marker=dict(color='#9694e0') 
            ),
            row=1, col=2
        )
        fig.add_trace(
            go.Scatterpolar(
                r=avg_salary_men.tolist()+[avg_salary_men.tolist()[0]], theta=occupation.tolist()+[occupation.tolist()[0]], visible=visible_indicator, fill='toself', fillcolor='rgba(70, 145, 216, 0.36)', line=dict(color='#4793d8', shape='spline'), name='Men'
            ),
            row=1, col=1
        )
        fig.add_trace(
            go.Scatterpolar(
                r=avg_salary_women.tolist()+[avg_salary_women.tolist()[0]], theta=occupation.tolist()+[occupation.tolist()[0]], visible=visible_indicator, fill='toself', fillcolor='rgba(217, 91, 74, 0.36)', line=dict(color='#d85847', shape='spline'), name='Women'
            ),
            row=1, col=1
        )
        
        if (label in occupation.tolist()) and (i==k):
            fig.add_trace(
                go.Scatterpolar(
                    r=[avg_salary_men[occupation.tolist().index(label)]], theta=[label], visible=visible_indicator, fill='toself', fillcolor='rgba(70, 145, 216, 0.36)', 
                    name='Men', marker=dict(size=30, color='rgba(70, 145, 216, 0.16)'), line=dict(color='#4793d8', width=10, shape='spline'), showlegend=False
                ),
                row=1, col=1
            )
            fig.add_trace(
                go.Scatterpolar(
                    r=[avg_salary_women[occupation.tolist().index(label)]], theta=[label], visible=visible_indicator, fill='toself', fillcolor='rgba(70, 145, 216, 0.36)', 
                    name='Women', marker=dict(size=30, color='rgba(217, 91, 74, 0.16)'), line=dict(color='#4793d8', width=10, shape='spline'), showlegend=False
                ),
                row=1, col=1
            )
            fig.add_trace(
                go.Bar(
                    x=[label], y=[pay_gap_percent[occupation.tolist().index(label)]], name=Category[i], visible=visible_indicator, base='stack', marker=dict(color='rgba(250, 0, 0, 0.3)'), showlegend=False
                ),
                row=1, col=2
            )
        else:
            fig.add_trace(
                go.Scatterpolar(
                    r=[np.nan], theta=[np.nan], visible=visible_indicator, fill='toself', fillcolor='rgba(70, 145, 216, 0.36)', 
                    name='Men', marker=dict(size=30, color='rgba(70, 145, 216, 0.16)'), line=dict(color='#4793d8', width=10, shape='spline'), showlegend=False
                ),
                row=1, col=1
            )
            fig.add_trace(
                go.Scatterpolar(
                    r=[np.nan], theta=[np.nan], visible=visible_indicator, fill='toself', fillcolor='rgba(70, 145, 216, 0.36)', 
                    name='Women', marker=dict(size=30, color='rgba(217, 91, 74, 0.16)'), line=dict(color='#4793d8', width=10, shape='spline'), showlegend=False
                ),
                row=1, col=1
            )
            fig.add_trace(
                go.Bar(
                    x=[np.nan], y=[np.nan], name=Category[i], visible=visible_indicator, base='stack', marker=dict(color='rgba(250, 0, 0, 0.3)'), showlegend=False
                ),
                row=1, col=2
            )

        buttons_dict = dict()
        buttons_dict['label'] = Category[i]
        buttons_dict['method'] = "update"
        visible = [False] * len(Category) * 6
        # visible[i] = True
        visible[(i + 1) * 6 - 6] = True
        visible[(i + 1) * 6 - 5] = True
        visible[(i + 1) * 6 - 4] = True
        visible[(i + 1) * 6 - 3] = True
        visible[(i + 1) * 6 - 2] = True
        visible[(i + 1) * 6 - 1] = True
        buttons_dict['args'] = [{'visible': visible}]
        buttons.append(buttons_dict)
    
    fig.update_layout(

        yaxis=dict(
        tickformat='.0%'),
        barmode='stack',
        updatemenus=[
            dict(
                direction="down",
                pad={"r": 10, "t": 10},  # PADDING
                x=0.8,  # POSITION
                y=1.27,
                xanchor="left",  # ANCHOR POINT
                yanchor="top",
                buttons=buttons,
                active=k
            )
        ],
        autosize=False,
        paper_bgcolor='#f0eeeb',
        plot_bgcolor='#f0eeeb',
        polar=dict(bgcolor='#f0eeeb',
                   radialaxis=dict(gridcolor='#d5d5d4', ticks='', #range=[0, 130000],
                                   tickfont=dict(color='#545454', size=12)),
                   angularaxis=dict(linecolor='#d5d5d4', gridwidth=2, tickcolor='#ffffff'),
                   ),
                   
        title={
            'text': '<b>' + "Gender Pay Gap by Occupation in the US (2016)" + '</b>',
            'x': 0.5,  # Set x position to center
            'y': 0.95,  # Set y position to top
            'xanchor': 'center',  # Set x anchor to center
            'yanchor': 'top',  # Set y anchor to top
            'font': {'size': 18}  # Set font size and bold property
        },
        annotations=[dict(text='<b>' +"The percentage difference in wage between men and women"+'<b>' , x=1.06, y=-0.5, font_size=14, showarrow=False, xref="paper", yref="paper", align="center")],



        
        font=dict(
            family="Arial",
            size=15,
        ),
        legend=dict(
            orientation="h",
            yanchor="top",
            y=1.25,
            xanchor="center",
            x=0.5
        ),
        margin={"t": 150, "b": 200, 'r': 50, 'l': 50},
        height=700,
        width=1000,
        
        )
    
    return fig

fig = Figure(None, 0)
#fig.write_html('demo.html')
fig.show()

# Plot2 Diversity in different fields

In [3]:
df = pd.read_csv("../../data/cleaned/Diversity_in_different_fields_clean.csv")


In [4]:

# Create the horizontal stacked bar plot
fig = go.Figure(data=[go.Bar(y=df['Occupation'], x=df['Female'].values/100, 
                             name="Female", orientation='h', 
                             marker=dict(color='#e08c7e')), 
                      go.Bar(y=df['Occupation'], x=df['Male'].values/100, 
                             name="Male", orientation='h', 
                             marker=dict(color='#89a9e0'))], 
                layout=go.Layout(barmode='stack'))

fig.update_layout( 
    title={
        'text': '<b>'+"Gender Distribution Across Occupations"+'</b>',
        'x': 0.5,  # Set x position to center
        'y': 1,  # Set y position to top
        'xanchor': 'center',  # Set x anchor to center
        'yanchor': 'top',  # Set y anchor to top
        'font': {'size': 20}  # Set font size and bold property
    },
    xaxis=dict(
        tickformat='.0%',
    ),
    yaxis_title='<b>'+'Occupations'+'<b>', 
    xaxis_title='<b>'+'Percentage'+'<b>', 
    autosize=False, 
    height=750,
    width=820, 
    paper_bgcolor='#f0eeeb',
    plot_bgcolor='#f0eeeb',
    font=dict(family="Arial",size=15,),
    legend=dict(
        orientation="h",
        yanchor="top",
        y=1.08,
        xanchor="center",
        x=0.25
    ),
    margin={"t":80,"b":100},
    template="simple_white"

)

fig.show()


# Plot 1 Dash app publish

In [5]:
import dash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
from jupyter_dash import JupyterDash

fig = Figure(None, 0)
K = 0
l = ""

# Build App
app = JupyterDash(__name__)
app.layout = dash.html.Div(
    [dash.dcc.Graph(id="graph", figure=fig)]
)

@app.callback(
    Output("graph", "figure"),
    Input("graph", "clickData"),
)
def click(clickData):
    global K
    global l
    if not clickData:
        raise dash.exceptions.PreventUpdate
    try:
        r = {k: clickData["points"][0][k] for k in ["x", "y"]}
        for i in range(len(Category)):
            visible_indicator = False
            occupation = data_sub[data_sub['Category'] == Category[i]]['Occupation'].values
            if r['x'] in occupation:
                K = i
        fig = Figure(r['x'], K)
        l = r['x']
        return fig
    except:
        fig = Figure(l, K)
        return fig

# Run app and display result inline in the notebook
app.run_server(mode="inline")

Dash is running on http://127.0.0.1:8050/



# Plot 2 occupation table

In [1]:
import plotly.graph_objects as go
import pandas as pd

df = pd.read_excel("../../data/cleaned/Occupation_table.xlsx")

buttons = []
fig = go.Figure()
n = len(df.groupby("group"))
k = 0
rowEvenColor = '#ecebf0'
rowOddColor = 'white'
for i, j in df.groupby("group"):
    name = j.iloc[0]["data"]
    j = j.iloc[1:]
    fig.add_trace(go.Table(
        header=dict(values=[name], fill_color='#C2D4FF', height=36, font_size=20),
        cells=dict(values=[j.T.values[0].tolist()], fill = dict(color='#F5F8FF'),height=25, font_size=16, align=['center'],fill_color = [[rowOddColor,rowEvenColor]*(len(df)//2)])
    ))
    buttons_dict = dict()
    buttons_dict['label'] = name
    buttons_dict['method'] = "update"
    visible = [False] * n
    visible[k] = True
    buttons_dict['args'] = [{'visible': visible}]
    buttons.append(buttons_dict)
    k = k+1

fig.update_layout(
    plot_bgcolor='#f0eeeb',paper_bgcolor='#f0eeeb',
    updatemenus=[
        dict(
            direction="down",
            pad={"r": 5, "t": 5},  # PADDING
            x=0,  # POSITION
            y=1.1,
            xanchor="left",  # ANCHOR POINT
            yanchor="top",
            buttons=buttons,
            active=n-1
        )
    ],
    margin={"t": 10, "b": 20, 'r': 10, 'l': 20}
)

fig.show()